# Register the Chef into the Database #

data/images/known_faces has chef images. The names of the file is CHEFID_FNAME-LNAME.jpg . So we need to extract chef-id, name, image_path from the folder for all chefs. Then we need to get the encodings using face_recognition. Later we will connect to the database and add all these as entries to the database.

## 1. Imports ##

In [6]:
import os
import sqlite3
import face_recognition
import numpy as np
from datetime import datetime
from PIL import Image

## 2. Define Helper Functions ##

In [7]:
def extract_info_from_filename(filename):
    base = os.path.splitext(filename)[0]
    if '_' in base:
        chef_id, name = base.split('_', 1)
    else:
        chef_id, name = "Unknown", base
    return chef_id, name.replace('_', ' ')

## 3. Encode Image and Save to DB ##

In [8]:
db_path = '../data/chefs.db'

def connect_db(db_path='../data/chefs.db'):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    return conn, cursor

In [9]:
conn, cursor = connect_db(db_path)

In [10]:
def register_chef(image_path, db_path='../data/chefs.db'):
    print(f"Registering chef from image: {image_path}")

    image = face_recognition.load_image_file(image_path)
    encodings = face_recognition.face_encodings(image)
    
    if not encodings:
        print(f"❌ No face found in image: {image_path}")
        return
    
    encoding = encodings[0]
    chef_id, name = extract_info_from_filename(os.path.basename(image_path))

    cursor.execute("""
        INSERT INTO registered_chefs (chef_ID, name, image_path, encoding, timestamp)
        VALUES (?, ?, ?, ?, ?)
    """, (
        chef_id,
        name,
        image_path,
        str(list(encoding)),  # store encoding as string
        datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    ))
    
    conn.commit()
    print(f"✅ Registered: {name} ({chef_id})")

## 4. Register All Chefs in Folder ##

In [11]:
def register_all_known_faces(folder_path='../data/images/known_faces'):
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.jpg', '.png', '.jpeg')):
            full_path = os.path.join(folder_path, filename)
            register_chef(full_path)

In [12]:
register_all_known_faces('../data/images/known_faces')

Registering chef from image: ../data/images/known_faces\C0010_Harpal-Sokhi.jpg
✅ Registered: Harpal-Sokhi (C0010)
Registering chef from image: ../data/images/known_faces\C0012_Rohit-Sharma.jpg
✅ Registered: Rohit-Sharma (C0012)
Registering chef from image: ../data/images/known_faces\C001_Sanjeev-Kapoor.jpg
✅ Registered: Sanjeev-Kapoor (C001)
Registering chef from image: ../data/images/known_faces\C002_Vikas-Khanna.jpg
✅ Registered: Vikas-Khanna (C002)
Registering chef from image: ../data/images/known_faces\C003_Ranveer-Brar.jpg
✅ Registered: Ranveer-Brar (C003)
Registering chef from image: ../data/images/known_faces\C004_Madhur-Jaffrey.jpg
✅ Registered: Madhur-Jaffrey (C004)
Registering chef from image: ../data/images/known_faces\C005_Anjum-Anand.jpg
✅ Registered: Anjum-Anand (C005)
Registering chef from image: ../data/images/known_faces\C006_Hari-Nayak.jpg
✅ Registered: Hari-Nayak (C006)
Registering chef from image: ../data/images/known_faces\C007_Tarla-Dalal.jpg
✅ Registered: Tarla-D

In [13]:
conn.close()